In [28]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import urllib.parse

hostname = '#include ur localhost name'
database = '#include ur Database name'
username = '#include ur username'
pwd = '#include your password'
port_id = 5432

try:
    # Establish the connection using psycopg2
    conn = psycopg2.connect(
        host=hostname,
        dbname=database,
        user=username,
        password=pwd,
        port=port_id)

    print("Connection successful")

    # URL encode the password
    encoded_password = urllib.parse.quote(pwd)

    # Create the SQLAlchemy engine
    connection_string = f'postgresql://{username}:{encoded_password}@{hostname}:{port_id}/{database}'
    engine = create_engine(connection_string)

except Exception as error:
    print("Connection failed")
    print(error)



Connection successful


In [36]:
# List tables in the database
from sqlalchemy import inspect
inspector = inspect(engine)
tables = inspector.get_table_names()
print("Tables in the database:", tables)

 # View the contents of a specific table, e.g., 'grocery_sales'
table_name = 'grocery_sales'  # Replace with your actual table name
df = pd.read_sql_table(table_name, con=engine)

# Display the first few rows of the table
print(df.head())

grocery_sales = pd.read_sql_table('grocery_sales', con=engine)
print(grocery_sales)


Tables in the database: ['grocery_sales']
   index  Store_ID                     Date  Dept  Weekly_Sales
0      0         1  2010-02-05T00:00:00.000     1      24924.50
1      1         1  2010-02-05T00:00:00.000    26      11737.12
2      2         1  2010-02-05T00:00:00.000    17      13223.76
3      3         1  2010-02-05T00:00:00.000    45         37.44
4      4         1  2010-02-05T00:00:00.000    28       1085.29
       index  Store_ID                     Date  Dept  Weekly_Sales
0          0         1  2010-02-05T00:00:00.000     1      24924.50
1          1         1  2010-02-05T00:00:00.000    26      11737.12
2          2         1  2010-02-05T00:00:00.000    17      13223.76
3          3         1  2010-02-05T00:00:00.000    45         37.44
4          4         1  2010-02-05T00:00:00.000    28       1085.29
...      ...       ...                      ...   ...           ...
19995  20034         2  2012-09-14T00:00:00.000    11      16955.50
19996  20035         2  2012-0

In [41]:
import pandas as pd

# Load extra data from parquet file
extra_data = pd.read_parquet(r'C:\Users\12145\Documents\SkillTrack\Data Camp Project\extra_data.parquet')
print(extra_data)


         index  IsHoliday  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0            0          0        42.31       2.572        0.0        0.0   
1            1          0        42.31       2.572        0.0        0.0   
2            2          0        42.31       2.572        0.0        0.0   
3            3          0        42.31       2.572        0.0        0.0   
4            4          0        42.31       2.572        0.0        0.0   
...        ...        ...          ...         ...        ...        ...   
231517  232414          0        55.75       4.192        0.0        0.0   
231518  232415          0        55.75       4.192        0.0        0.0   
231519  232416          0        55.75       4.192        0.0        0.0   
231520  232417          0        55.75       4.192        0.0        0.0   
231521  232418          0        55.75       4.192        0.0        0.0   

        MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  Type  \
0           

In [43]:
print("grocery_sales columns:", grocery_sales.columns)
print("extra_data columns:", extra_data.columns)

grocery_sales columns: Index(['index', 'Store_ID', 'Date', 'Dept', 'Weekly_Sales'], dtype='object')
extra_data columns: Index(['index', 'IsHoliday', 'Temperature', 'Fuel_Price', 'MarkDown1',
       'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI',
       'Unemployment', 'Type', 'Size'],
      dtype='object')


In [44]:
grocery_sales['Month'] = pd.to_datetime(grocery_sales['Date']).dt.month
grocery_sales = grocery_sales[['Store_ID', 'Month', 'Dept', 'Weekly_Sales']]

if 'IsHoliday' in extra_data.columns and 'CPI' in extra_data.columns and 'Unemployment' in extra_data.columns:
        extra_data = extra_data[['IsHoliday', 'CPI', 'Unemployment']]
        
        # Concatenate the DataFrames
        combined_data = pd.concat([grocery_sales, extra_data], axis=1)

        # Save the combined data to a CSV file
        combined_data.to_csv('clean_data.csv', index=False)
        print("clean_data saved successfully.")
else:
        print("Required columns are missing in extra_data.")

clean_data saved successfully.


In [51]:
clean_data = combined_data[['Store_ID', 'Month', 'Dept', 'IsHoliday', 'Weekly_Sales', 'CPI', 'Unemployment']]


In [53]:
clean_data.to_csv('clean_data.csv', index=False)
print("clean_data saved successfully.")

clean_data saved successfully.


In [54]:
agg_data = clean_data.groupby('Month').agg({
    'Weekly_Sales': 'sum'
}).reset_index()

In [55]:
agg_data.to_csv('agg_data.csv', index=False)
print("agg_data saved successfully.")
   

agg_data saved successfully.


In [57]:
# Save the combined data to a CSV file
clean_data.to_csv('clean_data.csv', index=False)
print("clean_data saved successfully.")

# Group by Month and aggregate Weekly_Sales
agg_data = clean_data.groupby('Month').agg({
    'Weekly_Sales': 'sum'
}).reset_index()

# Save agg_data to a CSV file
agg_data.to_csv('agg_data.csv', index=False)
print("agg_data saved successfully.")

clean_data saved successfully.
agg_data saved successfully.


In [56]:
print(agg_data)

    Month  Weekly_Sales
0     1.0   25069663.31
1     2.0   43268092.94
2     3.0   45050672.57
3     4.0   47503671.89
4     5.0   41200198.89
5     6.0   45628898.43
6     7.0   46169757.33
7     8.0   44753664.10
8     9.0   37511541.35
9    10.0   35575334.65
10   11.0   30219256.35
11   12.0   41775566.37
